In [3]:
import os
import torch
import pandas as pd

In [24]:
class EpilepsyDataset:
    def __init__(self, path_to_data: str):
        self.path = path_to_data
        self.folders_with_patients = os.listdir(self.path)
        
        self.patients_data = []
        for patient in self.folders_with_patients:
            self.patients_data.append(os.listdir(os.path.join(self.path + patient)))
        
    def get_patients_data(self, patient: str) -> list[str]:
        return os.listdir(os.path.join(self.path, patient))
    
    def get_patients_names(self) -> list[str]:
        return os.listdir(self.path)
    
    def get_all_sensors_records_for_patient(self, patient: str) -> list[str]:
        if patient not in self.get_patients_names():
            raise ValueError('There is no such name')
         
        full_path_to_patient = os.path.join(self.path, patient) 
        
        return [name for name in os.listdir(full_path_to_patient) 
                if os.path.isdir(os.path.join(full_path_to_patient, name)) 
                and name[0] != '.']
    
    def _get_sorted_segments(self, sensor_folder: str) -> list[list]:
        """
        Get types of sensors from folder and sort its data by number of segment
        """
        
        delimiter = '_'
        full_path = os.path.join(self.path, sensor_folder)
        sensors_files_names = [file.split(delimiter) for file in os.listdir(full_path)]

        cropped_sensors_files_names = [file[:-2] for file in sensors_files_names]
        
        index_of_parameters = 3
        unique_parameters = set(file[index_of_parameters] for file in cropped_sensors_files_names)

        return_data = []
        for unique_item in unique_parameters:
            data = [file for file in sensors_files_names if unique_item in file]
            index_of_segments_number = 5
            sorted_list = sorted(data, key=lambda x: int(x[index_of_segments_number].split('.')[0]))
            return_data.append(['_'.join(file) for file in sorted_list])
    
        return return_data

In [25]:
dataset = EpilepsyDataset('/workspace/data_seerpy/data_seerpy/data/')

# dataset._get_sorted_segments('MSEL_00172/Empatica-ACC/')

print(dataset.get_all_sensors_records_for_patient('MSEL_00172'))

['Empatica-EDA', 'Empatica-BVP', 'Byteflies-accY', 'Empatica-HR', 'Byteflies-emg', 'Byteflies-accX', 'Byteflies-accZ', 'Empatica-ACC', 'Empatica-TEMP']
